Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [0]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [0]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [0]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### Two situations
- logistic regression with l2 loss function
    - train_subset = 10000
    - beta = 0.01
    - num_steps = 801
- neural network with l2 loss function
    - batch_size = 128
    - beta = 0.01
    - num_steps = 3001

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

#### Two situations
- log reg with l2 loss and small training data
    - train_subset = 100
    - beta = 0.01
    - num_steps = 801
- neural network wiht l2 loss and small training data
    - batch_size = 128
    - beta = 0.01
    - num_steps = 3001
    - train_dataset_2 = train_dataset[:500, :]

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

##### add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


- trying smaller beta and bigger num steps
    - batch_size = 128
    - beta = 0.0001
    - num_steps = 6001
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    - loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    - loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )
- trying keep prob of 0.1
    - batch_size = 128
    - beta = 0.001
    - num_steps = 3001
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.1}
- trying with learning rate and multiple keep probs
    - batch_size = 128
    - beta = 0.001
    - num_steps = 3001
    - kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp}
    - learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.95, staircase=True)
    - optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
- trying different learning rates
    - batch_size = 128
    - beta = 0.001
    - num_steps = 3001
    - lr in np.arange(0.0001, 0.001, 0.0001).tolist()
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, learnr : lr}
- trying 2 hideen layer neural network
    - batch_size = 128
    - beta = 0.001
    - hidden_nodes1 = 1024
    - hidden_nodes2 = 512
    - num_steps = 3001
    - kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]
    - lr in np.arange(0.0001, 0.001, 0.0001).tolist()
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
- trying 2 NN with different loss function
    - batch_size = 128
    - beta = 0.001
    - hidden_nodes1 = 1024
    - hidden_nodes2 = 512
    - num_steps = 3001
    - kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]
    - lr in np.arange(0.0001, 0.001, 0.0001).tolist()
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
    - loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    - loss = tf.reduce_mean( loss + beta * ( tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_weights2)))
- trying NN with 5 hidden layers
    - batch_size = 128
    - beta = 0.001
    - hidden_nodes1 = 1024
    - hidden_nodes2 = 512
    - hidden_nodes3 = 256
    - hidden_nodes4 = 128
    - hidden_nodes5 = 64
    - num_steps = 3001
    - kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]
    - lr in np.arange(0.0001, 0.001, 0.0001).tolist()
    - feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
    - loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    - loss = loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights))